In [1]:
%pip install datasets

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 5.1 MB/s eta 0:00:004.2 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.2/213.2 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.3 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.9/757.9 kB 5.8 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.3/197.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 4.6 MB/s eta 0:00:00
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.

In [9]:
from datasets import load_dataset

dataset_artificial = load_dataset("pubmed_qa", "pqa_artificial")

dataset_labeled = load_dataset("pubmed_qa", "pqa_labeled")

Found cached dataset pubmed_qa (/home/max/.cache/huggingface/datasets/pubmed_qa/pqa_artificial/1.0.0/dd4c39f031a958c7e782595fa4dd1b1330484e8bbadd4d9212e5046f27e68924)
100%|██████████| 1/1 [00:00<00:00, 172.13it/s]
Found cached dataset pubmed_qa (/home/max/.cache/huggingface/datasets/pubmed_qa/pqa_labeled/1.0.0/dd4c39f031a958c7e782595fa4dd1b1330484e8bbadd4d9212e5046f27e68924)
100%|██████████| 1/1 [00:00<00:00, 529.99it/s]


In [12]:
print(dataset_labeled)

print(dataset_labeled['train'][0])

DatasetDict({
    train: Dataset({
        features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
        num_rows: 1000
    })
})
{'pubid': 21645374, 'question': 'Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?', 'context': {'contexts': ['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.', 'The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole wi

In [27]:
#format the data
import json

def format_dataset_forqlora(dataset):
    qlora_dataset = {
        'data': []
    }

    for example in dataset['train']:
        paragraphs = []
        context = example['context']['contexts'][0]
        paragraph = {
            'context': context,
            'qas': []
        }
        paragraphs.append(paragraph)

        question = example['question']
        answer = example['long_answer']
        qas = {
            'question': question,
            'id': str(example['pubid']),
            'answers': [
                {
                    'text': answer,
                    'answer_start': context.find(answer)
                }
            ]
        }
        paragraphs[0]['qas'].append(qas)

        data = {
            'title': f"Document {example['pubid']}",
            'paragraphs': paragraphs
        }
        qlora_dataset['data'].append(data)

    return qlora_dataset

# Example usage
qlora_formatted_dataset = format_dataset_forqlora(dataset_labeled)

# Save the formatted dataset to a file
with open('qlora_dataset.json', 'w') as f:
    json.dump(qlora_formatted_dataset, f)
